In [13]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np
import bocd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import yfinance as yf

from machine_learning_finance import (analyze_trades, 
    plot_backtest_analysis, RangeTrainingWindowUtil,
    metrics_to_dataframe, TraderEnv, make_price_marker_from_boolean, add_marker, add_line)

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from datetime import datetime, timedelta

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "GOOG"
file = f"../backtests/{symbol}-model-back-test.csv"
period = 365
pd.options.display.max_rows = None
inverse = None # set to anything to graph inverse longs

def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()


        
def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)    
    test_df = calc_probabilties_without_lookahead(test_df, hist_df, hist_cutoff="2023-05-15")
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    now = datetime.now()
    start = now - timedelta(days=365)
    start = start.strftime('%Y-%m-%d')
    end = now.strftime('%Y-%m-%d')
    training_window = RangeTrainingWindowUtil(df, start, end)
    env = TraderEnv(symbol, training_window.test_df, training_window.hist_df)
    
    
    fig = plot_backtest_analysis(env.orig_timeseries_with_analytics, ledger, inverse=inverse)

    # convert each change point column to price of the average in question
    analytics_df = env.orig_timeseries_with_analytics
    periods=[30, 60, 90]
    colors=["blue", "purple", "green"]
    for period, color in zip(periods, colors):        
        marker_col =  f"cp-loc-{period}"
        trend_col = f"trend-{period}"
        analytics_df = make_price_marker_from_boolean(analytics_df, f"change-point-{period}", f"trend-{period}", marker_col) 
        add_marker(fig, analytics_df, marker_col, f"change-point-{period}", "diamond", 8, color)
        add_line(fig, analytics_df, trend_col, f"moving-avg-{period}", color)

    analytics_df["expert"] = env.expert_actions
    analytics_df["expert_buy"] =  np.where(analytics_df["expert"] == 1, analytics_df["Close"], np.nan)
    analytics_df["expert_sell"] =  np.where(analytics_df["expert"] == 2, analytics_df["Close"], np.nan)
    add_marker(fig, analytics_df, "expert_buy", "expert_buy", "triangle-up", 8, "springgreen")
    add_marker(fig, analytics_df, "expert_sell", "expert_sell", "triangle-down", 8, "purple")

    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    fig.show()

    
    fig = make_subplots(rows=3, cols=1)
    indexes=[1,2,3]
    for period, color, index in zip(periods, colors, indexes):        
        fig.add_trace(go.Bar(x=analytics_df.index, y=analytics_df[f'polynomial_derivative-{period}'], marker=dict(color=f'{color}', colorscale='Viridis'), name=f'Polynomial Derivative {period}'), row=index, col=1)

    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    fig.show()

    return ledger, training_window.test_df, env, analytics_df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)


ledger, df2, env, analytics_df = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 
analytics_df
#df2[["moving_avg", "change_points", "rt", "Close"]]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,trend-30,trend-diff-30,change-point-30,polynomial_derivative-30,...,long_entry,long_exit,short_entry,short_exit,cp-loc-30,cp-loc-60,cp-loc-90,expert,expert_buy,expert_sell
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-25,108.879997,110.580002,107.010002,108.209999,108.209999,28289900,112.317634,-4.107635,False,0.219623,...,108.209999,NaN,NaN,NaN,NaN,NaN,NaN,1,108.209999,NaN
2022-07-26,107.430000,107.739998,104.760002,105.440002,105.440002,36626600,112.269750,-6.829748,False,0.216410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,105.440002,NaN
2022-07-27,109.599998,114.400002,108.419998,113.599998,113.599998,41474600,112.483284,1.116715,False,0.213197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,113.599998,NaN
2022-07-28,112.800003,114.699997,111.850998,114.589996,114.589996,23303800,112.623267,1.966729,False,0.209984,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,114.589996,NaN
2022-07-29,113.400002,116.900002,113.230003,116.639999,116.639999,31336200,112.956733,3.683266,False,0.206772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,116.639999
2022-08-01,115.529999,117.120003,114.690002,115.480003,115.480003,22856200,113.210550,2.269453,False,0.203559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,115.480003
2022-08-02,114.430000,117.080002,114.260002,115.900002,115.900002,17911000,113.340050,2.559951,False,0.200346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,115.900002
2022-08-03,116.339996,119.419998,116.150002,118.779999,118.779999,25302800,113.564917,5.215082,False,0.197133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,118.779999
2022-08-04,118.300003,119.500000,117.709999,118.870003,118.870003,15757700,113.771100,5.098902,False,0.193920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,118.870003


In [ ]:
ledger